In [1]:
import pandas as pd
import numpy as np

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
#konlpy 설치
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

--2022-09-30 06:00:35--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz [following]
--2022-09-30 06:00:36--  https://www.dropbox.com/s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc36ca78699d073972ee634c83f4.dl.dropboxusercontent.com/cd/0/get/Bt5_1-Jytwd2FT7rA_swyN5rQurSSeTi8ahhNc4KPD8i1Zf6iJgG92U9QXkQEIY2bhVwU9gL-AZE4WH5EpFBWj_u9R_VFPvP29wcC3CkS5ZH-nvfEd1gwIMmyOb---qK3lFumqpubD-cPyMEZMdp7q7Q8tvFYDBuJyxDqwPV7G1TSWx6vxlW7LUmhrUXwH1DZ50/file?dl=1# [following]
--2022-09-30 06:00:36--  https://uc36ca78699d073972ee634c83f4.dl.dropboxusercontent.com/cd/0/get/Bt5_1-Jytwd2

In [3]:
from konlpy.tag import Mecab, Okt

In [4]:
import re

def text_preprocessing(text_list):   #list 형태의 text_list를 입력받는 전처리 함수
    
    stopwords = ['을', '를', '이', '가', '은', '는', '.', '"', '!', '~', '게', '걸', '았'] #불용어 설정
    tokenizer = Mecab() #형태소 분석기 
    token_list = []
    
    for lyrics in text_list:   #바깥쪽 리스트

        for text in lyrics:    #안쪽 리스트
            txt = re.sub('[^가-힣]', ' ', text) #한글만 남기고 다른 글자 모두 제거
            token = tokenizer.morphs(txt) #형태소 분석
            token = [t for t in token if t not in stopwords] #형태소 분석 결과 중 stopwords에 해당하지 않는 것만 추출
            token_list.append(token)
        
    return token_list, tokenizer

In [5]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import os

lyrics_dir = "/content/drive/MyDrive/sample_data/nell_lyrics/"

In [7]:
filenames = os.listdir(lyrics_dir)

In [8]:
def text_load(base_dir, name):
    text_path = base_dir
    f = open(base_dir + name, 'r')

    lines = f.readlines()
    lines = [line.strip() for line in lines]
    
    f.close()
    return lines

In [9]:
all_lyrics = []

for filename in filenames:
    all_lyrics.append(text_load(lyrics_dir, filename))

In [10]:
all_lyrics[:2]

[['지금까진 전부 잊어',
  '조용히 두 눈을 감고, 널 가둔 그 벽을 부숴',
  '알고 있었던 모든게 아무것도 아닌게 될까,',
  '그렇게 될까봐 두려워?',
  'Just breathe in & breathe out your dreams with me.',
  '',
  'I’m in the ocean of light',
  '내 꿈이 숨을 쉰다',
  '눈부신 빛의 파도 그 속에서 새롭게 태어나고 있어',
  'I’m in the ocean of light',
  '내 꿈이 춤을 춘다',
  '거대한 빛의 파도 그 속에서 난 다시 태어나고 있어',
  'In the ocean of light',
  '',
  '새로워질 너를 믿어',
  '조용히 두 눈을 감고, 늘 꿈꿔왔던 너를 그려',
  '믿어왔었던 모든 게 아무것도 아닌 게 될까,',
  '그렇게 될까봐 두려워?',
  'Just breathe in & breathe out your dreams with me',
  '',
  'I’m in the ocean of light',
  '내 꿈이 숨을 쉰다',
  '눈부신 빛의 파도 그 속에서 새롭게 태어나고 있어',
  'I’m in the ocean of light',
  '내 꿈이 춤을 춘다',
  '거대한 빛의 파도 그 속에서 난 다시 태어나고 있어',
  'In the ocean of light',
  '',
  'I’m in the ocean of light',
  '내 꿈이 숨을 쉰다',
  '눈부신 빛의 파도 그 속에서 새롭게 태어나고 있어',
  'I’m in the ocean of light',
  '내 꿈이 춤을 춘다',
  '거대한 빛의 파도 그 속에서',
  '난 다시 태어나고 있어',
  'In the ocean of light',
  '',
  'In the ocean of light',
  'In the ocean of light',
  'In the ocean of light',
  "I'm in the ocean 

In [11]:
#리스트 내부 빈 문자열 제거
all_lyrics = [list(filter(None, lyrics)) for lyrics in all_lyrics]

In [12]:
all_lyrics[:2]

[['지금까진 전부 잊어',
  '조용히 두 눈을 감고, 널 가둔 그 벽을 부숴',
  '알고 있었던 모든게 아무것도 아닌게 될까,',
  '그렇게 될까봐 두려워?',
  'Just breathe in & breathe out your dreams with me.',
  'I’m in the ocean of light',
  '내 꿈이 숨을 쉰다',
  '눈부신 빛의 파도 그 속에서 새롭게 태어나고 있어',
  'I’m in the ocean of light',
  '내 꿈이 춤을 춘다',
  '거대한 빛의 파도 그 속에서 난 다시 태어나고 있어',
  'In the ocean of light',
  '새로워질 너를 믿어',
  '조용히 두 눈을 감고, 늘 꿈꿔왔던 너를 그려',
  '믿어왔었던 모든 게 아무것도 아닌 게 될까,',
  '그렇게 될까봐 두려워?',
  'Just breathe in & breathe out your dreams with me',
  'I’m in the ocean of light',
  '내 꿈이 숨을 쉰다',
  '눈부신 빛의 파도 그 속에서 새롭게 태어나고 있어',
  'I’m in the ocean of light',
  '내 꿈이 춤을 춘다',
  '거대한 빛의 파도 그 속에서 난 다시 태어나고 있어',
  'In the ocean of light',
  'I’m in the ocean of light',
  '내 꿈이 숨을 쉰다',
  '눈부신 빛의 파도 그 속에서 새롭게 태어나고 있어',
  'I’m in the ocean of light',
  '내 꿈이 춤을 춘다',
  '거대한 빛의 파도 그 속에서',
  '난 다시 태어나고 있어',
  'In the ocean of light',
  'In the ocean of light',
  'In the ocean of light',
  'In the ocean of light',
  "I'm in the ocean of light"],
 ['하루가 길었어',
  '고작

In [13]:
#가사 전체 토큰화
lyrics_tokenlist, mecab = text_preprocessing(all_lyrics)

In [14]:
lyrics_tokenlist[:50]

[['지금', '까진', '전부', '잊', '어'],
 ['조용히', '두', '눈', '감', '고', '널', '가둔', '그', '벽', '부숴'],
 ['알', '고', '있', '었', '던', '모든', '아무것', '도', '아닌게', '될까'],
 ['그렇', '될까봐', '두려워'],
 [],
 [],
 ['내', '꿈', '숨', '쉰다'],
 ['눈부신', '빛', '의', '파도', '그', '속', '에서', '새롭', '태어나', '고', '있', '어'],
 [],
 ['내', '꿈', '춤', '춘다'],
 ['거대', '한', '빛', '의', '파도', '그', '속', '에서', '난', '다시', '태어나', '고', '있', '어'],
 [],
 ['새로워질', '너', '믿', '어'],
 ['조용히', '두', '눈', '감', '고', '늘', '꿈꿔왔', '던', '너', '그려'],
 ['믿', '어', '왔었', '던', '모든', '아무것', '도', '아닌', '될까'],
 ['그렇', '될까봐', '두려워'],
 [],
 [],
 ['내', '꿈', '숨', '쉰다'],
 ['눈부신', '빛', '의', '파도', '그', '속', '에서', '새롭', '태어나', '고', '있', '어'],
 [],
 ['내', '꿈', '춤', '춘다'],
 ['거대', '한', '빛', '의', '파도', '그', '속', '에서', '난', '다시', '태어나', '고', '있', '어'],
 [],
 [],
 ['내', '꿈', '숨', '쉰다'],
 ['눈부신', '빛', '의', '파도', '그', '속', '에서', '새롭', '태어나', '고', '있', '어'],
 [],
 ['내', '꿈', '춤', '춘다'],
 ['거대', '한', '빛', '의', '파도', '그', '속', '에서'],
 ['난', '다시', '태어나', '고', '있', '어'],
 [],
 [],
 [],
 [],
 [],


In [15]:
#토큰 리스트 내부 빈 리스트 제거
lyrics_tokenlist = list(filter(None, lyrics_tokenlist))

In [16]:
lyrics_tokenlist[:50]

[['지금', '까진', '전부', '잊', '어'],
 ['조용히', '두', '눈', '감', '고', '널', '가둔', '그', '벽', '부숴'],
 ['알', '고', '있', '었', '던', '모든', '아무것', '도', '아닌게', '될까'],
 ['그렇', '될까봐', '두려워'],
 ['내', '꿈', '숨', '쉰다'],
 ['눈부신', '빛', '의', '파도', '그', '속', '에서', '새롭', '태어나', '고', '있', '어'],
 ['내', '꿈', '춤', '춘다'],
 ['거대', '한', '빛', '의', '파도', '그', '속', '에서', '난', '다시', '태어나', '고', '있', '어'],
 ['새로워질', '너', '믿', '어'],
 ['조용히', '두', '눈', '감', '고', '늘', '꿈꿔왔', '던', '너', '그려'],
 ['믿', '어', '왔었', '던', '모든', '아무것', '도', '아닌', '될까'],
 ['그렇', '될까봐', '두려워'],
 ['내', '꿈', '숨', '쉰다'],
 ['눈부신', '빛', '의', '파도', '그', '속', '에서', '새롭', '태어나', '고', '있', '어'],
 ['내', '꿈', '춤', '춘다'],
 ['거대', '한', '빛', '의', '파도', '그', '속', '에서', '난', '다시', '태어나', '고', '있', '어'],
 ['내', '꿈', '숨', '쉰다'],
 ['눈부신', '빛', '의', '파도', '그', '속', '에서', '새롭', '태어나', '고', '있', '어'],
 ['내', '꿈', '춤', '춘다'],
 ['거대', '한', '빛', '의', '파도', '그', '속', '에서'],
 ['난', '다시', '태어나', '고', '있', '어'],
 ['하루', '길', '었', '어'],
 ['고작', '글자', '몇', '개'],
 ['난', '또', '그렇게', '망가져', 

In [17]:
#vocab size 불러오기

tokenizer_keras = Tokenizer()
tokenizer_keras.fit_on_texts(lyrics_tokenlist)
vocab_size = len(tokenizer_keras.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 2153


In [18]:
sequences = list()

for sentence in lyrics_tokenlist:

    # 각 샘플에 대한 정수 인코딩
    encoded = tokenizer_keras.texts_to_sequences([sentence])[0] 
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

In [19]:
index_to_word = {}
for key, value in tokenizer_keras.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 5번 단어 : {}'.format(index_to_word[5]))

빈도수 상위 5번 단어 : 해


In [20]:
max_len = max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 18


In [21]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  86 608]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  86 608 232]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0  86 608 232 107]]


In [22]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1] #맨 우측 단어만 레이블로 분리

In [23]:
print(X[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  86]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  86 608]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0  86 608 232]]


In [24]:
y = to_categorical(y, num_classes=vocab_size)  #레이블 데이터 y에 대해서 원-핫 인코딩을 수행 (loss: categorical_crossentropy)

In [25]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [26]:
#모델 설계

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras import regularizers

In [32]:
#다중 클래스 분류 모델 만들기 (마지막 시점에서 모든 가능한 단어 중 하나의 단어를 예측)
#하이퍼파라미터 - 임베딩 벡터의 차원은 15, 은닉 상태의 크기는 128

# model = Sequential()
# model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
# model.add(Bidirectional(LSTM(150, return_sequences = True)))
# model.add(Dropout(0.2))
# model.add(LSTM(100))
# model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
# model.add(Dense(total_words, activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# print(model.summary())

from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor = 'accuracy', patience = 4, mode = 'max')

embedding_dim = 15
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len-1))
model.add(Bidirectional(LSTM(150, return_sequences = True)))
model.add(Dropout(0.2))
model.add(LSTM(hidden_units))
model.add(Dense(vocab_size/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=300, verbose=2, callbacks = [early_stopping])

Epoch 1/300
486/486 - 8s - loss: 6.4812 - accuracy: 0.0256 - 8s/epoch - 17ms/step
Epoch 2/300
486/486 - 5s - loss: 6.0183 - accuracy: 0.0253 - 5s/epoch - 9ms/step
Epoch 3/300
486/486 - 4s - loss: 5.9085 - accuracy: 0.0362 - 4s/epoch - 9ms/step
Epoch 4/300
486/486 - 4s - loss: 5.7573 - accuracy: 0.0561 - 4s/epoch - 9ms/step
Epoch 5/300
486/486 - 4s - loss: 5.6033 - accuracy: 0.0720 - 4s/epoch - 9ms/step
Epoch 6/300
486/486 - 4s - loss: 5.4828 - accuracy: 0.0895 - 4s/epoch - 9ms/step
Epoch 7/300
486/486 - 4s - loss: 5.3874 - accuracy: 0.1003 - 4s/epoch - 9ms/step
Epoch 8/300
486/486 - 4s - loss: 5.3071 - accuracy: 0.1064 - 4s/epoch - 9ms/step
Epoch 9/300
486/486 - 4s - loss: 5.2394 - accuracy: 0.1104 - 4s/epoch - 9ms/step
Epoch 10/300
486/486 - 4s - loss: 5.1836 - accuracy: 0.1147 - 4s/epoch - 9ms/step
Epoch 11/300
486/486 - 5s - loss: 5.1380 - accuracy: 0.1219 - 5s/epoch - 9ms/step
Epoch 12/300
486/486 - 5s - loss: 5.1004 - accuracy: 0.1215 - 5s/epoch - 9ms/step
Epoch 13/300
486/486 - 5

In [28]:
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre')

        # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

In [29]:
tokenizer_keras.word_index.items()

dict_items([('고', 1), ('어', 2), ('의', 3), ('에', 4), ('해', 5), ('지', 6), ('내', 7), ('있', 8), ('나', 9), ('없', 10), ('도', 11), ('그', 12), ('너', 13), ('하', 14), ('수', 15), ('말', 16), ('아', 17), ('한', 18), ('던', 19), ('난', 20), ('만', 21), ('것', 22), ('날', 23), ('더', 24), ('다', 25), ('않', 26), ('었', 27), ('안', 28), ('같', 29), ('싶', 30), ('겠', 31), ('들', 32), ('로', 33), ('할', 34), ('마음', 35), ('했', 36), ('속', 37), ('너무', 38), ('눈물', 39), ('시간', 40), ('지만', 41), ('또', 42), ('생각', 43), ('듯', 44), ('기억', 45), ('니', 46), ('면', 47), ('죠', 48), ('거', 49), ('우리', 50), ('널', 51), ('줘', 52), ('과', 53), ('으로', 54), ('때', 55), ('보', 56), ('네', 57), ('떠나', 58), ('모든', 59), ('그런', 60), ('알', 61), ('이렇게', 62), ('처럼', 63), ('되', 64), ('기', 65), ('야', 66), ('까지', 67), ('함', 68), ('참', 69), ('마', 70), ('라도', 71), ('주', 72), ('조금', 73), ('다시', 74), ('번', 75), ('일', 76), ('라고', 77), ('에서', 78), ('길', 79), ('엔', 80), ('행복', 81), ('나요', 82), ('버린', 83), ('오늘', 84), ('는데', 85), ('지금', 86), ('그저', 87), ('순간', 88), 

In [30]:
#임의의 단어로 시작하는 문장 생성

print(sentence_generation(model, tokenizer_keras, '눈물', 50))

눈물 의 위선 에 목 졸린 채 로 바다 만들 어 고 있 어 너 의 기술 과 잠든 이러 다 잠든 하 겠 줘 날 이젠 조금 더 그리워 하 될지 고 야 맘 의 연속 길 바랬 지 한없이 더러워 길 바라 겠 죠 적어도 너 와 어디 있


In [31]:
print(sentence_generation(model, tokenizer_keras, '꿈', 50))

꿈 꾸 렴 서부터 바라보 고 있 거 맞 죠 우리 쯤 에 있 어 내 마음 줘 내 손안 에 살 아 한단 모습 해 주 고 있 어 너 의 몰상식 함 에 필요 해 질 그 개 몇 개 의 아름다움 에 꼬리 어 모든 달 든 이렇
